# 0. Preamble

## 0.1. Connect to the Google shared folder "Capstone"

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/Capstone/'

/content/drive/My Drive/Capstone


In [3]:
ls

 chroma_database/         final_eval_details.gsheet
 chroma_test/            'GitHub related'/
 credentials/             Notebooks/
 datasets/                {persist_base_directory}vector_store_rcts_1500_small/
 Deliverables/            response_eval_accuracy.csv
 eval/                    response_eval_details.csv
 final_eval_details.csv


## 0.2. Install Dependencies

please install langchain required libraries: reference documentation  https://python.langchain.com/docs/how_to/installation/
<br> you will need your Open AI API key  https://platform.openai.com/api-keys


In [4]:
!pip install langchain
!pip install langchain-community
!pip install langchain-experimental
!pip install pypdf
!pip install unstructured
!pip install lxml html5lib
!pip install langchain-openai
!pip install chromadb
!pip install openai
!pip install -U langchain-chroma
!pip install colbert
!pip install groq
!pip install --upgrade langchain openai
!pip install transformers -U



# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.
# tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.

# Dependency Conflict: tensorflow and tensorflow-metadata require a specific version of protobuf (<5 and >=3.20.3), but the version installed (protobuf 5.29.0) does not meet this requirement.
# Successful Installation: Despite the warning, chromadb and its dependencies were installed successfully.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/

In [5]:
# install if using GPU

!pip install transformers torch accelerate bitsandbytes
!pip install -U bitsandbytes
!pip install --upgrade transformers bitsandbytes accelerate

# !pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


## 0.3. Imports

In [6]:
from dotenv import load_dotenv
import os
import time
import re
import math
import pickle
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gspread
import shutil
from sklearn.preprocessing import normalize

import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig

import openai

from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain.embeddings.base import Embeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

from langchain.schema import SystemMessage, HumanMessage

import groq
from groq import Client, Groq

# from google.colab import userdata

## 0.4. API Keys

<font color=red>delete below cell when share</font>

In [ ]:
# replace with your credentials
env_content = """
GROQ_API_KEY=
OPENAI_API_KEY=
HF_TOKEN=
"""

with open('credentials/claire.env', 'w') as f:   # replace it with your env filename
    f.write(env_content)

In [7]:
load_dotenv('credentials/claire.env', override=True) # replace it with your env filename

groq_api_key = os.getenv('GROQ_API_KEY')
huggingface_key = os.getenv('HF_TOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

<br>

# 1. Load the Optimal Vector Store

In [9]:
embedding_small = OpenAIEmbeddings(model ="text-embedding-3-small")

vector_store = Chroma(
    persist_directory="chroma_database/vector_store_rcts_1500_small",
    embedding_function=embedding_small
)

<br>

# 2. Chat bot with RAG

## 2.1. Optimal LLM

In [ ]:
# max_tokens = 4096  # Limit the number of tokens in responses
# temperature = 0 # Controls response randomness; The lower the temperature, the more focused the distribution becomes on the most likely tokens

# gpt3 = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=temperature,
#     max_tokens=max_tokens,
# )


# gpt4 = ChatOpenAI(
#     model="gpt-4",
#     temperature=temperature,
#     max_tokens=max_tokens,
# )

# gpt4o = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=temperature,
#     max_tokens=max_tokens,
# )

In [10]:
groq_client = Groq(api_key=groq_api_key)

In [11]:
# Create a class or function that mimics the interface of ChatOpenAI for LlaMa

class LlamaChat:
    def __init__(self, client, model, temperature=0, max_tokens=4096):
        """
        Initialize the LlamaChat wrapper.

        Parameters:
            client: Groq client instance.
            model (str): Llama model name.
            temperature (float): Sampling temperature.
            max_tokens (int): Maximum token limit for responses.
        """
        self.client = client
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens

    def invoke(self, messages):
        """
        Mimics the ChatOpenAI `invoke` method for Llama, handling both LangChain-style messages
        and single-string prompts.

        Parameters:
            messages (list or str): Either a list of LangChain messages or a single string prompt.

        Returns:
            A response object with a `.content` attribute for compatibility.
        """
        # Handle single-string prompts
        if isinstance(messages, str):
            prompt = messages
        else:
            # Convert LangChain messages to a single prompt
            prompt = "\n".join([msg.content for msg in messages])

        # Call the Llama model via Groq
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )

        # Extract the content of the response using attribute access
        class Response:
            def __init__(self, content):
                self.content = content

        return Response(content=response.choices[0].message.content)


In [12]:
# llama3_1_70b = LlamaChat(
#     client=groq_client,
#     model="llama-3.1-70b-versatile",
#     temperature=0,
#     max_tokens=4096,
# )

llama3_1_8b = LlamaChat(
    client=groq_client,
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=4096,
)


## 2.2. System Prompt & RAG LLM Response

In [13]:
system_prompt = """
You are a financial expert. Use the following context from SEC filings, news, or company presentations to answer the question accurately and concisely.
Use metadata to extract dates and return answers from the most recent dates.
If you don't know the answer, say you don't know.
"""

In [14]:
def rag_response(query=None, system_prompt=system_prompt, rag_model=llama3_1_8b, vector_store=vector_store, top_k=3):
    """
    Generates an answer using the ChatOpenAI model with context retrieved from a vector store.

    Parameters:
        model (ChatOpenAI): The initialized ChatOpenAI model.
        vector_store (Chroma): The vector store to retrieve relevant context.
        system_prompt (str): The system's instructions to the model.
        query (str): The user query.
        top_k (int): The number of top relevant documents to retrieve from the vector store.

    Returns:
        str: The model's response.
    """
    if query:

      # Step 1: Retrieve relevant context from the vector store
      docs = vector_store.similarity_search(query, k=top_k)
      context = "\n".join([doc.page_content for doc in docs])

      # Step 2: Construct the messages
      messages = [
          SystemMessage(content=system_prompt),
          HumanMessage(content=f"Context:\n{context}\n\nQuestion: {query}")
      ]

      # Step 3: Get the response from the model
      response = rag_model.invoke(messages)

      return response.content

    else: print('please enter a query')

## 2.3. Quick tests

In [15]:
# test one query - use llama

query = "What is Nvidia's revenue for their last quarter?"

rag_response(query)

"NVIDIA's revenue for their last quarter (Q3 Fiscal 2025) is $35.1 billion."

In [16]:
query =  "What are the largest suppliers of Nvidia?"

rag_response(query)

'Based on the provided context, I do not have information about the largest suppliers of Nvidia.'